In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!nvidia-smi

Mon Jan 27 17:13:17 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

Loading the required dataset.

In [0]:

import pandas as pd
import numpy as np

weather = pd.read_csv('gdrive/My Drive/Weather_data.csv')


In [61]:
weather.head(3)

,datetime_utc,_conds,_dewptm,_fog,_hail,_heatindexm,_hum,_precipm,_pressurem,_rain,_snow,_tempm,_thunder,_tornado,_vism,_wdird,_wdire,_wgustm,_windchillm,_wspdm
0,19961101-11:00,Smoke,9.0,0,0,NaN,27.0,NaN,1010.0,0,0,30.0,0,0,5.0,280.0,West,NaN,NaN,7.4
1,19961101-12:00,Smoke,10.0,0,0,NaN,32.0,NaN,-9999.0,0,0,28.0,0,0,NaN,0.0,North,NaN,NaN,NaN
2,19961101-13:00,Smoke,11.0,0,0,NaN,44.0,NaN,-9999.0,0,0,24.0,0,0,NaN,0.0,North,NaN,NaN,NaN


In [0]:
cols = weather.columns.tolist()

Lets clean the column names for easier retrieval.

In [0]:
for i in range(len(cols)):
  if cols[i][1]=="_":
    cols[i] = cols[i][2:]


In [0]:
weather.columns = cols

In [65]:
weather.head(3)

,datetime_utc,conds,dewptm,fog,hail,heatindexm,hum,precipm,pressurem,rain,snow,tempm,thunder,tornado,vism,wdird,wdire,wgustm,windchillm,wspdm
0,19961101-11:00,Smoke,9.0,0,0,NaN,27.0,NaN,1010.0,0,0,30.0,0,0,5.0,280.0,West,NaN,NaN,7.4
1,19961101-12:00,Smoke,10.0,0,0,NaN,32.0,NaN,-9999.0,0,0,28.0,0,0,NaN,0.0,North,NaN,NaN,NaN
2,19961101-13:00,Smoke,11.0,0,0,NaN,44.0,NaN,-9999.0,0,0,24.0,0,0,NaN,0.0,North,NaN,NaN,NaN


Checking null values, looks like many columns have null values.

In [66]:
weather.columns[weather.isna().any()]

Index(['conds', 'dewptm', 'heatindexm', 'hum', 'precipm', 'pressurem', 'tempm',
       'vism', 'wdird', 'wdire', 'wgustm', 'windchillm', 'wspdm'],
      dtype='object')

In [67]:
weather.describe()

,dewptm,fog,hail,heatindexm,hum,precipm,pressurem,rain,snow,tempm,thunder,tornado,vism,wdird,wgustm,windchillm,wspdm
count,98294.000000,98913.000000,98913.000000,29111.000000,98160.000000,0.0,9.868200e+04,98913.000000,98913.00000,98244.000000,98913.000000,98913.000000,94497.000000,84533.000000,1063.000000,573.000000,96560.000000
mean,15.817130,0.067109,0.000131,35.659606,57.774358,NaN,1.951861e+03,0.026498,0.00001,25.578427,0.009362,0.000020,2.419885,162.550980,37.704516,5.708202,7.685854
std,7.096917,0.250213,0.011464,5.013579,23.754463,NaN,3.217101e+05,0.160612,0.00318,8.464732,0.096303,0.004497,20.970678,120.019889,6.841070,1.205967,11.995965
min,-24.000000,0.000000,0.000000,26.800000,4.000000,NaN,-9.999000e+03,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,25.900000,2.100000,0.000000
25%,10.000000,0.000000,0.000000,31.700000,39.000000,NaN,1.002000e+03,0.000000,0.00000,19.000000,0.000000,0.000000,1.500000,50.000000,33.300000,4.900000,0.000000
50%,15.000000,0.000000,0.000000,35.100000,58.000000,NaN,1.008000e+03,0.000000,0.00000,27.000000,0.000000,0.000000,2.000000,150.000000,37.000000,6.100000,7.400000
75%,22.000000,0.000000,0.000000,39.200000,77.000000,NaN,1.014000e+03,0.000000,0.00000,32.000000,0.000000,0.000000,3.000000,270.000000,40.700000,6.800000,11.100000
max,75.000000,1.000000,1.000000,73.600000,243.000000,NaN,1.010614e+08,1.000000,1.00000,90.000000,1.000000,1.000000,6436.000000,960.000000,92.600000,7.300000,1514.900000



*   Since 'tempm' is our target column and if the target column itself is null, then the corresponding samples are irrelevant.
*   And precipm column has all Na values. So we are dropping it. 




In [0]:
weather = weather.dropna(axis=0, subset=['tempm'])
weather = weather.drop(["precipm"],axis=1)

Lets have a look at the 'conds' column (The condition during the day).

In [69]:
from collections import Counter
Counter(weather.conds.values)

Counter({'Blowing Sand': 357,
         'Clear': 3111,
         'Drizzle': 109,
         'Fog': 2511,
         'Funnel Cloud': 2,
         'Haze': 46654,
         'Heavy Fog': 407,
         'Heavy Rain': 28,
         'Heavy Thunderstorms and Rain': 18,
         'Heavy Thunderstorms with Hail': 1,
         'Light Drizzle': 406,
         'Light Fog': 51,
         'Light Hail Showers': 1,
         'Light Haze': 4,
         'Light Rain': 1291,
         'Light Rain Showers': 5,
         'Light Sandstorm': 6,
         'Light Thunderstorm': 63,
         'Light Thunderstorms and Rain': 169,
         'Mist': 8977,
         'Mostly Cloudy': 1515,
         'Overcast': 311,
         'Partial Fog': 958,
         'Partly Cloudy': 2052,
         'Patches of Fog': 813,
         'Rain': 389,
         'Rain Showers': 2,
         'Sandstorm': 1,
         'Scattered Clouds': 2182,
         'Shallow Fog': 1842,
         'Smoke': 20200,
         'Squalls': 6,
         'Thunderstorm': 189,
         'Thunderst

If we inspect our dataset closely, we notice that we have columns such as 'fog', 'hail','rain','snow','thunder','tornado'.
These columns are similar to a one hot encoded one. So we must find a method to group other conditions so that they fall under these main (lets call them master conditions) conditions.

In [0]:
def generalize_conditions(dataset):   
  weather_conds = dataset.conds.values.tolist()
  for i in range(len(weather_conds)):
    if type(weather_conds[i])!=float:
      if weather_conds[i].__contains__("Hail"):
        weather_conds[i] = "hail"
      elif weather_conds[i].__contains__("Thunder") or weather_conds[i].__contains__("Squalls") or weather_conds[i].__contains__("Cloud") or weather_conds[i].__contains__("Overcast"):
        weather_conds[i] = "thunder"
      elif weather_conds[i].__contains__("Rain") or weather_conds[i].__contains__("Drizzle") or weather_conds[i].__contains__("Showers"):
        weather_conds[i] = "rain"
      elif weather_conds[i].__contains__("Mist") or weather_conds[i].__contains__("Showers"):
        weather_conds[i] = "snow"
      elif weather_conds[i].__contains__("Fog"):
        weather_conds[i] = "fog"
      elif weather_conds[i].__contains__("Sand") or weather_conds[i].__contains__("Haze") or weather_conds[i].__contains__("Dust") or weather_conds[i].__contains__("Smoke") or weather_conds[i].__contains__("Ash"):
        weather_conds[i] = "haze"
  dataset["conds"] = weather_conds

generalize_conditions(weather)

We have cleaned the conds column.

In [71]:
Counter(weather.conds.values)

Counter({'Clear': 3111,
         'Unknown': 276,
         'fog': 6582,
         'hail': 13,
         'haze': 70042,
         nan: 40,
         'rain': 2230,
         'snow': 8977,
         'thunder': 6973})

In [72]:
weather.columns[weather.isna().any()]

Index(['conds', 'dewptm', 'heatindexm', 'hum', 'pressurem', 'vism', 'wdird',
       'wdire', 'wgustm', 'windchillm', 'wspdm'],
      dtype='object')

Lets fill 'conds' column. How we do that is:


1.   We separate our dataset into two, one which has no null value in the conds column and the other which has only null values in conds column.
2.   We make use of this non-null dataset to fill dewptm column. 
3.   Next we extract the max dew and min dew & max vism and min vism for every condition.
4.   Next we extract the dew and vism columns from the null datset.
5.   Based on the range of dew and vism, we will impute values.



In [0]:
def convert_na_to_list(dew,dew_range_max,dew_range_min,vism,vism_range_max,vism_range_min):  
  li = []
  for i in range(len(dew)):
    if dew[i] >= dew_range_min[0] and dew[i] <= dew_range_max[0] and vism[i] >= vism_range_min[0] and vism[i] <= vism_range_max[0]:
      li.append('Clear')
    elif dew[i] >= dew_range_min[1] and dew[i] <= dew_range_max[1] and vism[i] >= vism_range_min[1] and vism[i] <= vism_range_max[1]:
      li.append('Unknown')
    elif dew[i] >= dew_range_min[4] and dew[i] <= dew_range_max[4] and vism[i] >= vism_range_min[4] and vism[i] <= vism_range_max[4]:
      li.append('haze')
  return li 

In [74]:
def weather_na(weather):
  weather_conds_wo_na = weather[~weather["conds"].isna()]
  weather_conds_w_na = weather[weather["conds"].isna()]
  weather_conds_wo_na["dewptm"] = weather_conds_wo_na.groupby("conds").transform(lambda x: x.fillna(x.median()))
  weather_conds_wo_na["dewptm"] = weather_conds_wo_na['dewptm'].fillna(weather_conds_wo_na.groupby('conds')['dewptm'].transform('mean'))
  dew_range_max, dew_range_min = weather_conds_wo_na.groupby('conds')['dewptm'].max(), weather_conds_wo_na.groupby('conds')['dewptm'].min()
  vism_range_max, vism_range_min = weather_conds_wo_na.groupby('conds')['vism'].max(),weather_conds_wo_na.groupby('conds')['vism'].min()
  dew = weather_conds_w_na.dewptm.values
  vism = weather_conds_w_na.vism.values
  list1 = convert_na_to_list(dew,dew_range_max,dew_range_min,vism,vism_range_max,vism_range_min)
  weather_conds_w_na['conds'] = list1
  weather_conds_w_na["dewptm"] = weather_conds_w_na['dewptm'].fillna(weather_conds_w_na.groupby('conds')['dewptm'].transform('mean'))
  weather = weather_conds_wo_na.append(weather_conds_w_na, ignore_index=True)
  weather = weather.sort_values('datetime_utc')
  return weather
weather = weather_na(weather)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

Now lets get one hot encoded columns based on the conds, whilst also retaining conds column itself for future imputations.

In [0]:

conds = weather['conds'].values

In [0]:
weather = weather.drop(['fog','hail','thunder','tornado','rain','snow'], axis=1)
weather = pd.get_dummies(weather, columns=['conds'])
weather['conds'] = conds

In [77]:
weather.columns[weather.isna().any()]

Index(['heatindexm', 'hum', 'pressurem', 'vism', 'wdird', 'wdire', 'wgustm',
       'windchillm', 'wspdm'],
      dtype='object')

Further inspection of the dataset shows that the wdire affects the wdird. So lets impute wdird for the dataset split which doesnt have a null value in wdire column. 

In [0]:
weather_windDir_wo_na = weather[~weather["wdire"].isna()]
weather_windDir_w_na = weather[weather["wdire"].isna()]


In [0]:
def convert_na_dird_to_list(dird,list_titles,list_max_min):  
  li = []
  print(len(dird))
  for i in range(len(dew)):
    for j in range(17):
      if dird[i] >= list_max_min[j][0] and dird[i] <= list_max_min[j][1]:
        li.append(list_titles[j])
        break
  return li 

In [80]:
def weather_wind_direction_na(weather):
  weather_windDir_wo_na = weather[~weather["wdire"].isna()]
  weather_windDir_w_na = weather[weather["wdire"].isna()]
  weather_windDir_wo_na["wdird"] = weather_windDir_wo_na['wdird'].fillna(weather_windDir_wo_na.groupby('wdire')['wdird'].transform('mean'))
  weather = weather_windDir_wo_na.append(weather_windDir_w_na, ignore_index=True)
  weather = weather.sort_values('datetime_utc')
  return weather

weather = weather_wind_direction_na(weather)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


A method to fill null values. 

In [0]:
def fill_null(col,group_by_col,method):
  weather[col] = weather[col].fillna(weather.groupby(group_by_col)[col].transform(method))

Fill null values of certain columns based on conds column.

In [0]:
fill_null("vism","conds","mean")
fill_null("heatindexm","conds","mean")
fill_null("hum","conds","mean")
fill_null("vism","conds","mean")
weather["heatindexm"] = weather.groupby("conds").transform(lambda x: x.fillna(x.mean()))

# weather['vism'] = weather['vism'].fillna(weather.groupby('conds')['vism'].transform('mean'))


In [83]:
weather.columns[weather.isna().any()]

Index(['pressurem', 'wdird', 'wdire', 'wgustm', 'windchillm', 'wspdm'], dtype='object')

Lets obtain the mean value of windchillm per direction.

In [0]:
wd = weather[~weather.windchillm.isna()].wdire.values.tolist()
wc = weather[~weather.windchillm.isna()].windchillm.values.tolist()


In [0]:
dic = dict()
for i in range(len(wd)):
  if wd[i] in dic.keys():
    dic[wd[i]].append(wc[i])
  else:
    dic[wd[i]] = [wc[i]]

In [86]:
avgDict = {}
for k,v in dic.items():
    avgDict[k] = sum(v)/ float(len(v))
avgDict

{'ENE': 6.3125,
 'ESE': 6.85,
 'East': 6.492857142857142,
 'NE': 6.375000000000001,
 'NNE': 6.966666666666666,
 'NNW': 5.557142857142857,
 'NW': 5.664285714285713,
 'North': 6.18,
 'SE': 6.8375,
 'SSE': 6.225,
 'SSW': 6.05294117647059,
 'SW': 5.880434782608697,
 'South': 7.133333333333333,
 'WNW': 5.442553191489362,
 'WSW': 5.474626865671644,
 'West': 5.527956989247307}

In [0]:
avgDict["Variable"] = 10

Lets impute the values of wdir.

In [0]:
weather_windDir_wo_na = weather[~weather["wdire"].isna()]
weather_windDir_w_na = weather[weather["wdire"].isna()]


In [0]:
wdir1 = weather_windDir_wo_na.wdire.values.tolist()
wdir1 = [avgDict[i] for i in wdir1]

In [90]:
weather_windDir_wo_na["windchillm"] = wdir1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [0]:
weather = weather_windDir_wo_na.append(weather_windDir_w_na, ignore_index=True)
weather = weather.sort_values('datetime_utc')

Imputing further columns.

In [0]:
fill_null("wspdm","conds","median")
fill_null("windchillm","conds","median")
fill_null("pressurem","conds","median")
fill_null("wdird","conds","median")


In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

Lets divise a strategy to impute wdire. This column is very important in our final model for regression. Just as much as the condition. 


1.   We will first tokenize every possible direction (word to number).
2.   We will then impute based on conds column again. 
3.   We will revert these tokens back to words.



In [94]:
from sklearn.preprocessing import LabelEncoder
item_enc = LabelEncoder()
# le.inverse_transform(df[x])
weather_windDir_wo_na = weather[~weather["wdire"].isna()]
weather_windDir_w_na = weather[weather["wdire"].isna()]
weather_windDir_wo_na['wdire'] = item_enc.fit_transform(weather_windDir_wo_na['wdire'].values)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
weather = weather_windDir_wo_na.append(weather_windDir_w_na, ignore_index=True)
weather = weather.sort_values('datetime_utc')
weather.wdire = weather.wdire.apply(pd.to_numeric)

In [0]:
fill_null("wdire","conds","mean")
fill_null("wgustm","conds","mean")


In [0]:
weather.wdire = weather.wdire.astype(int)
wdir2 = weather.wdire.values.tolist()
wdir2 = [item_enc.inverse_transform([i])[0] for i in wdir2]
weather["wdire"] = wdir2

Now lets get the one hot encoded columns for wdire as well.

In [0]:
weather = pd.get_dummies(weather, columns=['wdire'])
weather = weather.drop(["conds"],axis=1)

In [0]:
weather = weather.drop(["datetime_utc"],axis=1)

Scale every column except the one hot encoded ones (conds and wdire)

In [0]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()

weather[['dewptm','heatindexm','hum','pressurem','vism','wdird','wgustm','windchillm','wspdm']] = mms.fit_transform(weather[['dewptm','heatindexm','hum','pressurem','vism','wdird','wgustm','windchillm','wspdm']])



In [101]:
weather.head(3)

,dewptm,heatindexm,hum,pressurem,tempm,vism,wdird,wgustm,windchillm,wspdm,conds_Clear,conds_Unknown,conds_fog,conds_hail,conds_haze,conds_rain,conds_snow,conds_thunder,wdire_ENE,wdire_ESE,wdire_East,wdire_NE,wdire_NNE,wdire_NNW,wdire_NW,wdire_North,wdire_SE,wdire_SSE,wdire_SSW,wdire_SW,wdire_South,wdire_Variable,wdire_WNW,wdire_WSW,wdire_West
0,0.559322,0.559322,0.096234,0.000109,30.0,0.000777,0.291667,0.171208,0.018739,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.576271,0.576271,0.117155,0.000000,28.0,0.000412,0.000000,0.171208,0.161811,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0.593220,0.593220,0.167364,0.000000,24.0,0.000412,0.000000,0.171208,0.161811,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [0]:
weather = weather.drop("wgustm",axis=1)

In [103]:
weather.columns[weather.isna().any()]

Index([], dtype='object')

In [104]:
weather.head(10)

,dewptm,heatindexm,hum,pressurem,tempm,vism,wdird,windchillm,wspdm,conds_Clear,conds_Unknown,conds_fog,conds_hail,conds_haze,conds_rain,conds_snow,conds_thunder,wdire_ENE,wdire_ESE,wdire_East,wdire_NE,wdire_NNE,wdire_NNW,wdire_NW,wdire_North,wdire_SE,wdire_SSE,wdire_SSW,wdire_SW,wdire_South,wdire_Variable,wdire_WNW,wdire_WSW,wdire_West
0,0.559322,0.559322,0.096234,0.000109,30.0,0.000777,0.291667,0.018739,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0.576271,0.576271,0.117155,0.000000,28.0,0.000412,0.000000,0.161811,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0.593220,0.593220,0.167364,0.000000,24.0,0.000412,0.000000,0.161811,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0.576271,0.576271,0.154812,0.000109,24.0,0.000311,0.000000,0.161811,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,0.593220,0.593220,0.179916,0.000109,23.0,0.000186,0.000000,0.161811,0.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
5,0.610169,0.610169,0.217573,0.000109,21.0,0.000412,0.000000,0.161811,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
6,0.627119,0.627119,0.234310,0.000109,21.0,0.000124,0.000000,0.161811,0.000000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
7,0.627119,0.627119,0.234310,0.000000,21.0,0.000412,0.000000,0.161811,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
8,0.627119,0.627119,0.267782,0.000000,19.0,0.000412,0.000000,0.161811,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
9,0.627119,0.627119,0.267782,0.000109,19.0,0.000412,0.000000,0.161811,0.004885,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


Ready to train and predict.

In [0]:
y = weather.tempm.values

X = weather.drop('tempm',axis=1)
X = X.values

In [0]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.33,random_state=2, shuffle=True)


This method will return the rmse, mse and the accuracy of our model.

In [0]:
def metric_display(preds):
  errors = abs(preds - y_test)
  final_rmse = np.sqrt(np.mean(errors*errors))
  mape = 100 * (errors / y_test)
  accuracy = 100 - np.mean(mape)
  return "Mean Absolute Error: ", round(np.mean(errors), 2), "RMSE: ", final_rmse, "Accuracy: ", round(accuracy, 2)


Model 1: LSTM

In [0]:
X_train_lst=X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test_lst=X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [0]:
X_train_lst.shape

(65823, 33, 1)

Custom function to calculate RMSE Loss

In [0]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 


In [0]:
from pandas import read_csv
from keras.models import Sequential, Model
from keras.layers import Dense, LSTM, Input, Flatten
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.callbacks import EarlyStopping


model = Sequential()
inp = Input(shape=(33,1))
x = LSTM(100, activation='relu', dropout=0.02)(inp)
x = Dense(1, kernel_initializer='normal')(x)
model = Model(inputs=inp, outputs=x)
model.compile(optimizer='adam', loss = root_mean_squared_error, metrics = ['mean_absolute_error'])
es = EarlyStopping(monitor='val_loss', mode='min')
model.fit(X_train_lst, y_train,batch_size=150,epochs=10, verbose=1)


Epoch 1/10
65823/65823 [==============================] - 25s 373us/step - loss: 11.2488 - mean_absolute_error: 9.6228
Epoch 2/10
65823/65823 [==============================] - 23s 345us/step - loss: 3.3286 - mean_absolute_error: 2.4437
Epoch 3/10
65823/65823 [==============================] - 22s 336us/step - loss: 2.8433 - mean_absolute_error: 2.0131
Epoch 4/10
65823/65823 [==============================] - 22s 340us/step - loss: 2.7823 - mean_absolute_error: 1.9950
Epoch 5/10
65823/65823 [==============================] - 23s 347us/step - loss: 2.5964 - mean_absolute_error: 1.8482
Epoch 6/10
65823/65823 [==============================] - 23s 350us/step - loss: 2.4866 - mean_absolute_error: 1.7906
Epoch 7/10
65823/65823 [==============================] - 23s 355us/step - loss: 2.2831 - mean_absolute_error: 1.5844
Epoch 8/10
65823/65823 [==============================] - 22s 340us/step - loss: 2.3022 - mean_absolute_error: 1.5547
Epoch 9/10
65823/65823 [==============================]

In [0]:
preds = model.predict(X_test_lst)

Scores.

In [0]:
preds = preds.tolist()
preds = [item for sublist in preds for item in sublist]
preds = np.array(preds)

scores_lstm = metric_display(preds)
print(scores_lstm)

('Mean Absolute Error: ', 1.34, 'RMSE: ', 1.7513295058210372, 'Accuracy: ', 94.72)


2. SVM

In [0]:
from sklearn.svm import SVR
svm=SVR()
svm.fit(X_train,y_train)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [0]:
preds_svm = svm.predict(X_test)
scores_svm = metric_display(preds_svm)
print(scores_svm)


('Mean Absolute Error: ', 1.08, 'RMSE: ', 1.8504668441381333, 'Accuracy: ', 95.76)


3. Random Forest.

In [0]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators= 100, random_state=42)
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [0]:
preds_rf = rf.predict(X_test)
scores_rf = metric_display(preds_rf)
print(scores_rf)

('Mean Absolute Error: ', 0.35, 'RMSE: ', 0.8697221326114547, 'Accuracy: ', 98.59)


4. Gradient Boost

In [0]:
from sklearn.ensemble import GradientBoostingRegressor
gb = GradientBoostingRegressor(n_estimators= 100, random_state=42)
gb.fit(X_train, y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=42, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [0]:
preds_gb = gb.predict(X_test)
scores_gb = metric_display(preds_gb)
print(scores_gb)

('Mean Absolute Error: ', 0.81, 'RMSE: ', 1.2352603173992993, 'Accuracy: ', 96.51)


5. Adaboost

In [0]:
from sklearn.ensemble import AdaBoostRegressor
ada = AdaBoostRegressor(n_estimators= 200, random_state=42)
ada.fit(X_train, y_train)

AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
                  n_estimators=200, random_state=42)

In [0]:
preds_ada = ada.predict(X_test)
scores_ada = metric_display(preds_ada)
print(scores_ada)


('Mean Absolute Error: ', 2.16, 'RMSE: ', 2.7442530228282207, 'Accuracy: ', 89.37)


6. ANN / MLP

In [0]:
model = Sequential()
model.add(Dense(50, input_dim=33, init='normal', activation='relu'))
model.add(Dense(20, init='normal', activation='relu'))
model.add(Dense(1, init='normal'))

model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train,y_train,batch_size=150,verbose=1,epochs=50)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, input_dim=33, activation="relu", kernel_initializer="normal")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(20, activation="relu", kernel_initializer="normal")`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal")`
  after removing the cwd from sys.path.


Epoch 1/50
65823/65823 [==============================] - 3s 42us/step - loss: 204.4446
Epoch 2/50
65823/65823 [==============================] - 1s 20us/step - loss: 35.2727
Epoch 3/50
65823/65823 [==============================] - 1s 19us/step - loss: 22.7668
Epoch 4/50
65823/65823 [==============================] - 1s 21us/step - loss: 9.5031
Epoch 5/50
65823/65823 [==============================] - 1s 20us/step - loss: 5.1762
Epoch 6/50
65823/65823 [==============================] - 1s 20us/step - loss: 4.6987
Epoch 7/50
65823/65823 [==============================] - 1s 21us/step - loss: 4.3106
Epoch 8/50
65823/65823 [==============================] - 1s 21us/step - loss: 3.8879
Epoch 9/50
65823/65823 [==============================] - 1s 20us/step - loss: 3.4252
Epoch 10/50
65823/65823 [==============================] - 1s 20us/step - loss: 3.0308
Epoch 11/50
65823/65823 [==============================] - 1s 21us/step - loss: 2.7290
Epoch 12/50
65823/65823 [=======================

In [0]:
preds_mlp = model.predict(X_test)
preds_mlp = preds_mlp.tolist()
preds_mlp = [item for sublist in preds_mlp for item in sublist]
preds_mlp = np.array(preds_mlp)

scores_mlp = metric_display(preds_mlp)
print(scores_mlp)

('Mean Absolute Error: ', 0.87, 'RMSE: ', 1.3878406644226562, 'Accuracy: ', 96.7)


Summarizing results.

In [0]:
dictionary = {"MAE":[],"RMSE":[],"Accuracy":[]}

for i in range(len(scores_ada)):
  if i==1:
    dictionary["MAE"].append(scores_ada[i])
    dictionary["MAE"].append(scores_gb[i])
    dictionary["MAE"].append(scores_lstm[i])
    dictionary["MAE"].append(scores_rf[i])
    dictionary["MAE"].append(scores_svm[i])
    dictionary["MAE"].append(scores_mlp[i])
  elif i==3:
    dictionary["RMSE"].append(scores_ada[i])
    dictionary["RMSE"].append(scores_gb[i])
    dictionary["RMSE"].append(scores_lstm[i])
    dictionary["RMSE"].append(scores_rf[i])
    dictionary["RMSE"].append(scores_svm[i])
    dictionary["RMSE"].append(scores_mlp[i])
  elif i==5:
    dictionary["Accuracy"].append(scores_ada[i])
    dictionary["Accuracy"].append(scores_gb[i])
    dictionary["Accuracy"].append(scores_lstm[i])
    dictionary["Accuracy"].append(scores_rf[i])
    dictionary["Accuracy"].append(scores_svm[i])
    dictionary["Accuracy"].append(scores_mlp[i])

dictionary

{'Accuracy': [89.37, 96.51, 94.72, 98.59, 95.76, 96.7],
 'MAE': [2.16, 0.81, 1.34, 0.35, 1.08, 0.87],
 'RMSE': [2.7442530228282207,
  1.2352603173992993,
  1.7513295058210372,
  0.8697221326114547,
  1.8504668441381333,
  1.3878406644226562]}

In [0]:
models = ["AdaBoost","GradientBoost","LSTM","RandomForest","SVM","MLP"]
result_df = pd.DataFrame.from_dict(dictionary,orient='index').transpose()
result_df["models"] = models
result_df = result_df[['models', 'MAE', 'RMSE', 'Accuracy',]]
result_df = result_df.sort_values('Accuracy')


Random Forest performs the best followed by ANN/MLP

In [0]:
result_df

,models,MAE,RMSE,Accuracy
0,AdaBoost,2.16,2.744253,89.37
2,LSTM,1.34,1.751330,94.72
4,SVM,1.08,1.850467,95.76
1,GradientBoost,0.81,1.235260,96.51
5,MLP,0.87,1.387841,96.70
3,RandomForest,0.35,0.869722,98.59
